In [2]:
# ==============================================================
# Imports necessários
# ==============================================================
from datetime import datetime
from uuid import uuid4
from enum import Enum

from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, Field, UUID4


# ==============================================================
# Inicialização da aplicação
# ==============================================================
app = FastAPI()


# ==============================================================
# Enum de Categorias
# ==============================================================
class CategoriaEnum(str, Enum):
    ELETRONICO = "Eletrônico"
    VESTUARIO = "Vestuário"
    ALIMENTO = "Alimento"
    OUTRO = "Outro"


# ==============================================================
# Modelo de Produto
# ==============================================================
class Produto(BaseModel):
    __produtos__ = []

    id: UUID4 = Field(default_factory=uuid4, description="ID do produto")
    nome: str = Field(..., min_length=2, description="Nome do produto")
    preco: float = Field(..., gt=0, description="Preço deve ser maior que zero")
    quantidade: int = Field(..., ge=0, description="Quantidade deve ser >= 0")
    categoria: CategoriaEnum = Field(..., description="Categoria do produto")
    data_cadastro: datetime = Field(default_factory=datetime.now, description="Data de cadastro")


# ==============================================================
# Rotas da API com tratamentos de erro
# ==============================================================
@app.get("/produtos", response_model=list[Produto])
async def listar_produtos():
    return list(Produto.__produtos__)


@app.post("/produtos", response_model=Produto)
async def cadastrar_produto(produto: Produto):
    # Erro 400: nome vazio
    if not produto.nome.strip():
        raise HTTPException(status_code=400, detail="Nome do produto não pode ser vazio")

    # Erro 409: produto duplicado
    for p in Produto.__produtos__:
        if p.nome.lower() == produto.nome.lower() and p.categoria == produto.categoria:
            raise HTTPException(
                status_code=409,
                detail=f"O produto '{produto.nome}' já existe na categoria '{produto.categoria}'"
            )

    Produto.__produtos__.append(produto)
    return produto


@app.get("/produtos/{produto_id}", response_model=Produto)
async def buscar_produto(produto_id: UUID4):
    try:
        return next((p for p in Produto.__produtos__ if p.id == produto_id))
    except StopIteration:
        raise HTTPException(status_code=404, detail="Produto não encontrado")


# ==============================================================
# Função de testes (sem 500)
# ==============================================================
def main():
    with TestClient(app) as client:

        print("Testando cadastro e duplicação...")
        produto_valido = {
            "nome": "Notebook",
            "preco": 3500.0,
            "quantidade": 5,
            "categoria": "Eletrônico"
        }
        r1 = client.post("/produtos", json=produto_valido)  # sucesso
        print("Sucesso:", r1.status_code, r1.json())

        r2 = client.post("/produtos", json=produto_valido)  # duplicado
        print("Erro duplicado:", r2.status_code, r2.json())

        print("\nTestando nome válido e inválido...")
        r3 = client.post("/produtos", json={
            "nome": "Camiseta",
            "preco": 59.9,
            "quantidade": 10,
            "categoria": "Vestuário"
        })  # sucesso
        print("Sucesso:", r3.status_code, r3.json())

        r4 = client.post("/produtos", json={
            "nome": "   ",  # nome vazio
            "preco": 10.0,
            "quantidade": 1,
            "categoria": "Vestuário"
        })  # erro
        print("Nome inválido:", r4.status_code, r4.json())

        print("\nTestando dados corretos e tipo errado...")
        r5 = client.post("/produtos", json={
            "nome": "Arroz 5kg",
            "preco": 25.0,
            "quantidade": 100,
            "categoria": "Alimento"
        })  # sucesso
        print("Sucesso:", r5.status_code, r5.json())

        r6 = client.post("/produtos", json={
            "nome": "Arroz ruim",
            "preco": "não é número",  # erro de tipo
            "quantidade": 50,
            "categoria": "Alimento"
        })  # erro
        print("Tipo inválido:", r6.status_code, r6.json())

        print("\nTestando busca existente e inexistente...")
        id_existente = r5.json()["id"]
        r7 = client.get(f"/produtos/{id_existente}")  # sucesso
        print("Produto encontrado:", r7.status_code, r7.json())

        from uuid import uuid4
        r8 = client.get(f"/produtos/{uuid4()}")  # inexistente
        print("Produto não encontrado:", r8.status_code, r8.json())


# ==============================================================
# Execução principal
# ==============================================================
if __name__ == "__main__":
    main()


Testando cadastro e duplicação...
Sucesso: 200 {'id': 'e795a436-d134-49a2-a2bd-11227e9ea371', 'nome': 'Notebook', 'preco': 3500.0, 'quantidade': 5, 'categoria': 'Eletrônico', 'data_cadastro': '2025-08-17T03:41:16.274261'}
Erro duplicado: 409 {'detail': "O produto 'Notebook' já existe na categoria 'CategoriaEnum.ELETRONICO'"}

Testando nome válido e inválido...
Sucesso: 200 {'id': 'a4180d13-8be2-4c0c-a109-a3a7a6e1832f', 'nome': 'Camiseta', 'preco': 59.9, 'quantidade': 10, 'categoria': 'Vestuário', 'data_cadastro': '2025-08-17T03:41:16.276295'}
Nome inválido: 400 {'detail': 'Nome do produto não pode ser vazio'}

Testando dados corretos e tipo errado...
Sucesso: 200 {'id': '7c8555f4-f56a-4cd6-a54f-0e775ef9c125', 'nome': 'Arroz 5kg', 'preco': 25.0, 'quantidade': 100, 'categoria': 'Alimento', 'data_cadastro': '2025-08-17T03:41:16.278912'}
Tipo inválido: 422 {'detail': [{'type': 'float_parsing', 'loc': ['body', 'preco'], 'msg': 'Input should be a valid number, unable to parse string as a num